In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [8]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
from langchain_community.vectorstores import Chroma

In [9]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'

###The output of  
pdf_enrichment_output_file = 'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'

In [10]:
# Load data (deserialized data) into serilaizable list of of data#####
###This acts as the source for the Graph Building ######
import pickle
with open(pdf_enrichment_output_dir+pdf_enrichment_output_file, 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [11]:
doc_len=len(document_dict_deserialized)
doc_len

479

In [12]:
document_dict_deserialized[0]

{'page': 89,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '57\nIntroducing pods\n Therefore, you need to run each process in its own container. That’s how Docker\nand Kubernetes are meant to be used. \n3.1.2\nUnderstanding pods\nBecause you’re not supposed to group multiple processes into a single container, it’s\nobvious you need another higher-level construct that will allow you to bind containers\ntogether and manage them as a single unit. This is the reasoning behind pods. \n A pod of containers allows you to run closely related processes together and pro-\nvide them with (almost) the same environment as if they were all running in a single\ncontainer, while keeping them somewhat isolated. This way, you get the best of both\nworlds. You can take advantage of all the features containers provide, while at the\nsame time giving the processes the illusion of running together. \nUNDERSTANDING THE PARTIAL ISOLATION BETWEEN CONTAINERS OF THE SAME POD\nIn the previous chapter, you learned t

In [13]:
vectorstore = Chroma(
    embedding_function=embeddings, persist_directory="./chroma_db_oai"
)


def load_vectorstore_chroma(db, embeddings, docs, persist=False):
    #db = Chroma(persist_directory="./chroma_db_oai", embedding_function=embeddings)
    ids = db.get()['ids']
    #print(ids)
    print(docs)

    if len(ids)>0:
        db.delete(
            ids=ids
        )
    db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db_oai")
    return db

# load from disk
def search_vectorstore_chroma(db,query):
    #db = Chroma(persist_directory="./chroma_db_oai", embedding_function=embeddings)
    docs = db.similarity_search(query,k=20)
    #retriever = db.as_retriever(k=20 , search_type="mmr", lambda_mult=.6)
    
    #docs = retriever.get_relevant_documents(query)
    return docs


# load from disk
def search_vectorstore_chroma_hybrid_retriever(db,embeddings,query):
    #db = Chroma(persist_directory="./chroma_db_oai", embedding_function=embeddings)

    retriever_sim = db.as_retriever(
    search_type="similarity", search_kwargs={"k": 20}
    )

    retriever_mmr = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 20}
    )   

    merger = MergerRetriever(retrievers=[retriever_sim, retriever_mmr])

    filter = EmbeddingsRedundantFilter(embeddings=embeddings)

    reordering = LongContextReorder()

    pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])

    compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=merger
    )

    docs=compression_retriever.get_relevant_documents(query)

    return docs

/home/matrix4284/mambaforge-pypy3/envs/search_agent_poc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [11]:

for idx in range(0,len(document_dict_deserialized)):
    document_dict_deserialized[idx]['entities']

[{'entity': 'Docker',
  'description': 'Containerization platform',
  'category': 'software'},
 {'entity': 'Kubernetes',
  'description': 'Container orchestration system',
  'category': 'software'},
 {'entity': 'Pods',
  'description': 'Group of containers that share resources and run as a single unit',
  'category': 'application'},
 {'entity': 'Containers',
  'description': 'Isolated execution environment for processes',
  'category': 'container'},
 {'entity': 'Linux namespaces',
  'description': 'Resource isolation mechanism in Linux',
  'category': 'process'},
 {'entity': 'Network namespace',
  'description': 'Shared network resources among containers in a pod',
  'category': 'network'},
 {'entity': 'UTS namespace',
  'description': 'Shared hostname and network interfaces among containers in a pod',
  'category': 'network'},
 {'entity': 'IPC namespace',
  'description': 'Shared inter-process communication resources among containers in a pod',
  'category': 'process'},
 {'entity': 'P

In [ ]:
def page_entity_description_embedder(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    page_text=document_dict_deserialized[page_idx]['entities']
    
    
    
    
    ########################################Summary Extraction Enrichment###########################################
    
    ##Summary Extraction Enrichment
    #page_summary_txt=extract_summary_per_page(page_text)
    
    #print("Page Summary Text")
    #print(page_summary_txt)
    
    ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
    #document_dict_deserialized_stage2[page_idx]['summary']=json.loads(page_summary_txt.strip())["summary"]
    
    ################################################################################################################
    
    ######################################Highlight Extraction######################################################
    
    ##Summary Extraction Enrichment
    return page_text
    ################################################################################################################